In [1]:
!pip install flask_ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd

from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify

from joblib import load
import requests

In [3]:
%run module.py

Загружаем модель:

In [4]:
model_response = requests.get('https://drive.google.com/uc?id=1VzLcNy_E7YEpbQRWPcVSWMwLiqQxacaU')
with open('GBC_model.joblib', 'wb') as file:
    file.write(model_response.content)
model = load('GBC_model.joblib')
model

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('workclass',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='workclass')),
                                                                 ('lower_strip',
                                                                  FunctionTransformer(func=<function lower_strip at 0x7f480b57ab80>)),
                                                                 ('ohe',
                                                                  OHEEncoder(key='workclass'))])),
                                                ('education',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='education')),
                                                                 ('lower_strip',
                                                                  F...
                                                                                     include_bias=False))])),
                                                ('hours-per-week',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='hours-per-week')),
                                                                 ('imputer',
                                                                  SimpleImputer(strategy='median')),
                                                                 ('scaler',
                                                                  StandardScaler()),
                                                                 ('poly',
                                                                  PolynomialFeatures(degree=5,
                                                                                     include_bias=False))]))])),
                ('classifier',
                 GradientBoostingClassifier(learning_rate=0.01, max_depth=10,
                                            n_estimators=300, random_state=5555,
                                            subsample=0.7))])

In [5]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz
!./ngrok authtoken 2OcKYO93mIfSooI1g5VjJiB9JYq_5XinmEyWDEapndN6gBtEK
!./ngrok http 80

--2023-04-19 21:34:03--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 52.202.168.65, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13856790 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz’

ngrok-stable-linux- 100%[===================>]  13.21M  6.51MB/s    in 2.0s    

2023-04-19 21:34:06 (6.51 MB/s) - ‘ngrok-stable-linux-amd64.tgz’ saved [13856790/13856790]

ngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


### **Создаем сервис для обработки запросов к модели**

In [6]:
app = Flask(__name__)
run_with_ngrok(app)  

@app.route('/', methods=['GET'])
def general():
    return 'Welcome to prediction process'

@app.route('/predict', methods=['POST'])
def predict():
    data = {'success': False}

    # ensure an image was properly uploaded to our endpoint
    age, workclass, fnlwgt, education, education_num = '', '', '', '', ''
    marital_status, occupation, relationship, race, sex = '', '', '', '', ''
    capital_gain, capital_loss, hours_per_week, native_country = '', '', '', ''

    request_json = request.get_json()
    
    if request_json['age'] or request_json['age'] == 0:
        age = request_json['age']
    if request_json['workclass']:
        workclass = request_json['workclass']
    if request_json['fnlwgt']:
        fnlwgt = request_json['fnlwgt']
    if request_json['education']:
        education = request_json['education']
    if request_json['education-num'] or request_json['education-num'] == 0:
        education_num = request_json['education-num']
    if request_json['marital-status']:
        marital_status = request_json['marital-status']
    if request_json['occupation']:
        occupation = request_json['occupation']
    if request_json['relationship']:
        relationship = request_json['relationship']
    if request_json['race']:
        race = request_json['race']
    if request_json['sex']:
        sex = request_json['sex']
    if request_json['capital-gain'] or request_json['capital-gain'] == 0:
        capital_gain = request_json['capital-gain']
    if request_json['capital-loss'] or request_json['capital-loss'] == 0:
        capital_loss = request_json['capital-loss']
    if request_json['hours-per-week'] or request_json['hours-per-week'] == 0:
        hours_per_week = request_json['hours-per-week']
    if request_json['native-country']:
        native_country = request_json['native-country']
    
    pr = pd.DataFrame({'age': [age], 'workclass': [workclass], 'fnlwgt': [fnlwgt], 
                       'education': [education], 'education-num': [education_num],
                       'marital-status': [marital_status], 'occupation': [occupation],
                       'relationship': [relationship], 'race': [race], 'sex': [sex],
                       'capital-gain': [capital_gain], 'capital-loss': [capital_loss],
                       'hours-per-week': [hours_per_week], 'native-country': [native_country]})
 
    preds = model.predict_proba(pr)
    data['predictions'] = preds[:, 1][0]
    data['description'] = f'{occupation}, {sex}, {age}, {education}'
    data['успех!'] = True
    print('OK')

    return jsonify(data)


if __name__ == '__main__':
    model=load('GBC_model.joblib')
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://2266-35-234-58-134.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [19/Apr/2023 21:34:31] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/2023 21:34:32] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/2023 21:35:02] "POST /predict HTTP/1.1" 200 -


OK


...

OK


INFO:werkzeug:127.0.0.1 - - [19/Apr/2023 21:35:53] "POST /predict HTTP/1.1" 200 -


OK


INFO:werkzeug:127.0.0.1 - - [19/Apr/2023 21:35:54] "POST /predict HTTP/1.1" 200 -


OK
